# 07. cuDF Import/Export and GPU Acceleration | استيراد/تصدير cuDF وتسريع GPU

## 📚 Learning Objectives

By completing this notebook, you will:
- Import and export data in different formats using cuDF functions
- Use GPU acceleration from cuDF to process data faster
- Compare cuDF performance with Pandas
- Optimize data processing using GPU acceleration

## 🔗 Prerequisites

- ✅ Examples 01–06 (data loading through EDA)
- ✅ Understanding of Pandas basics
- ✅ CUDA-capable GPU (optional, but recommended)

**Leads to:** Example 08 (Feature extraction); Unit 3, Unit 4.

---

## Official Structure Reference

This notebook covers practical activities from **Course 05, Unit 2**:
- Import/Export using cuDF: Importing and exporting data in different formats using cuDF functions
- Optimization using cuDF: Using GPU acceleration from cuDF to process data faster
- **Source:** `DETAILED_UNIT_DESCRIPTIONS.md` - Unit 2 Practical Content

---

## Introduction to cuDF

**cuDF** is a GPU-accelerated DataFrame library that provides a pandas-like API for working with data on GPUs. It's part of the RAPIDS ecosystem and can significantly speed up data processing operations.


🚀 Google Colab Setup (Run this first if using Colab)
دليل إعداد Google Colab (قم بتشغيل هذا أولاً إذا كنت تستخدم Colab)



In [2]:
# Try importing cuDF (requires CUDA and RAPIDS installation)
try:
    import cudf
    import numpy as np
    import pandas as pd
    CUDF_AVAILABLE = True
    print("✅ cuDF imported successfully!")
    print(f"cuDF version: {cudf.__version__}")
except ImportError:
    CUDF_AVAILABLE = False
    import numpy as np
    import pandas as pd
    print("⚠️  cuDF not available. Install RAPIDS for GPU acceleration:")
    print("   Note: Requires CUDA-capable GPU and RAPIDS installation")
    print("   Continuing with Pandas examples...")

print("✅ Libraries imported!")


⚠️  cuDF not available. Install RAPIDS for GPU acceleration:
   Note: Requires CUDA-capable GPU and RAPIDS installation
   Continuing with Pandas examples...
✅ Libraries imported!


## Part 1: Data Import with cuDF

cuDF supports importing data from CSV, Parquet, JSON, and other formats, similar to Pandas but with GPU acceleration.


In [3]:
# Generate sample data for demonstration
sample_data = pd.DataFrame({
    'id': range(10000), 'value1': np.random.randn(10000),
    'value2': np.random.randint(1, 100, 10000),
    'category': np.random.choice(['A', 'B', 'C'], 10000)
})

# Save to CSV for demonstration
sample_data.to_csv('sample_data_cudf.csv', index=False)
print("✅ Sample CSV file created")

if CUDF_AVAILABLE:
    # Import with cuDF (GPU-accelerated)
    print("\n" + "=" * 60)
    print("Importing with cuDF (GPU-accelerated):")
    print("=" * 60)
    df_cudf = cudf.read_csv('sample_data_cudf.csv')
    print(f"DataFrame shape: {df_cudf.shape}")
    print(f"\nFirst few rows:")
    print(df_cudf.head())
    
    # Import with Pandas (CPU)
    print("\n" + "=" * 60)
    print("Importing with Pandas (CPU):")
    print("=" * 60)
    df_pandas = pd.read_csv('sample_data_cudf.csv')
    print(f"DataFrame shape: {df_pandas.shape}")
else:
    # Demonstrate with Pandas
    print("\n" + "=" * 60)
    print("Importing with Pandas (CPU):")
    print("=" * 60)
    df_pandas = pd.read_csv('sample_data_cudf.csv')
    print(f"DataFrame shape: {df_pandas.shape}")
    print(f"\nFirst few rows:")
    print(df_pandas.head())


✅ Sample CSV file created

Importing with Pandas (CPU):
DataFrame shape: (10000, 4)

First few rows:
   id    value1  value2 category
0   0 -0.310255      12        A
1   1  0.283642      10        B
2   2  0.549608      28        B
3   3  0.368518      39        C
4   4 -0.154028      32        A


## Part 2: Data Export with cuDF

Exporting data to different formats (CSV, Parquet, JSON) using cuDF.


In [4]:
if CUDF_AVAILABLE:
    print("=" * 60)
    print("Exporting with cuDF:")
    print("=" * 60)
    
    # Export to CSV
    df_cudf.to_csv('output_cudf.csv', index=False)
    print("✅ Exported to CSV: output_cudf.csv")
    
    # Export to Parquet (efficient format, good for cuDF)
    df_cudf.to_parquet('output_cudf.parquet')
    print("✅ Exported to Parquet: output_cudf.parquet")
    
    # Note: cuDF also supports JSON, ORC, and other formats
    print("\nNote: cuDF supports CSV, Parquet, JSON, ORC, and other formats")
    print("Parquet format is often preferred for large datasets due to compression and speed")
else:
    # Demonstrate with Pandas
    print("=" * 60)
    print("Exporting with Pandas:")
    print("=" * 60)
    df_pandas.to_csv('output_pandas.csv', index=False)
    print("✅ Exported to CSV: output_pandas.csv")
    print("\nNote: Install cuDF/RAPIDS for GPU-accelerated export and Parquet support")


Exporting with Pandas:
✅ Exported to CSV: output_pandas.csv

Note: Install cuDF/RAPIDS for GPU-accelerated export and Parquet support


## Part 3: GPU Acceleration Performance Comparison

Let's compare cuDF (GPU) vs Pandas (CPU) performance for common operations.


In [5]:
import time

# Create larger dataset for performance comparison
large_data = pd.DataFrame({
    'id': range(100000), 'value1': np.random.randn(100000),
    'value2': np.random.randint(1, 1000, 100000),
    'category': np.random.choice(['A', 'B', 'C', 'D', 'E'], 100000)
})

if CUDF_AVAILABLE:
    # Convert to cuDF
    large_cudf = cudf.from_pandas(large_data)
    
    print("=" * 60)
    print("Performance Comparison: cuDF (GPU) vs Pandas (CPU)")
    print("=" * 60)
    
    # Operation 1: Filtering
    print("\n1. Filtering operations:")
    start = time.time()
    filtered_cudf = large_cudf[large_cudf['value2'] > 500]
    cudf_time = time.time() - start
    print(f"   cuDF (GPU): {cudf_time:.4f} seconds")
    
    start = time.time()
    filtered_pandas = large_data[large_data['value2'] > 500]
    pandas_time = time.time() - start
    print(f"   Pandas (CPU): {pandas_time:.4f} seconds")
    print(f"   Speedup: {pandas_time/cudf_time:.2f}x faster with cuDF")
    
    # Operation 2: Groupby
    print("\n2. Groupby operations:")
    start = time.time()
    grouped_cudf = large_cudf.groupby('category').mean()
    cudf_time = time.time() - start
    print(f"   cuDF (GPU): {cudf_time:.4f} seconds")
    
    start = time.time()
    grouped_pandas = large_data.groupby('category').mean()
    pandas_time = time.time() - start
    print(f"   Pandas (CPU): {pandas_time:.4f} seconds")
    print(f"   Speedup: {pandas_time/cudf_time:.2f}x faster with cuDF")
    
    print("\n✅ GPU acceleration provides significant speedup for large datasets!")
else:
    print("=" * 60)
    print("GPU Acceleration Benefits:")
    print("=" * 60)
    print("""
    cuDF (GPU-accelerated) provides:
    - 10-100x faster data processing for large datasets
    - Efficient memory usage on GPU
    - Seamless integration with other RAPIDS libraries
    - pandas-like API for easy migration
    
    Use cases:
    - Large-scale data cleaning and transformation
    - Real-time data processing pipelines
    - Accelerated exploratory data analysis
    
    Requirements:
    - CUDA-capable NVIDIA GPU
    - RAPIDS installation
    """)


GPU Acceleration Benefits:

    cuDF (GPU-accelerated) provides:
    - 10-100x faster data processing for large datasets
    - Efficient memory usage on GPU
    - Seamless integration with other RAPIDS libraries
    - pandas-like API for easy migration
    
    Use cases:
    - Large-scale data cleaning and transformation
    - Real-time data processing pipelines
    - Accelerated exploratory data analysis
    
    Requirements:
    - CUDA-capable NVIDIA GPU
    - RAPIDS installation
    


## Summary

### Key Concepts:
1. **cuDF**: GPU-accelerated DataFrame library with pandas-like API
2. **Import/Export**: Supports CSV, Parquet, JSON, ORC formats
3. **GPU Acceleration**: 10-100x faster for large datasets
4. **Use Cases**: Large-scale data processing, real-time pipelines, accelerated EDA

### Benefits:
- Significant performance improvements for large datasets
- Seamless pandas API compatibility
- Integration with RAPIDS ecosystem
- Efficient GPU memory utilization

**Reference:** Course 05, Unit 2: "Import/Export using cuDF" and "Optimization using cuDF: Using GPU acceleration"
